#📌 Extracción

## 🗂️ Importación de datos desde la API de Telecom X

In [3]:
import requests
import pandas as pd

In [4]:
# En este paso, cargamos los datos directamente desde la API publicada en GitHub.
# La fuente contiene información de clientes, incluyendo datos demográficos,
# tipos de servicios contratados y estado de cancelación del servicio (churn).

url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'

In [5]:
# Realizamos la solicitud HTTP a la URL
response = requests.get(url)

In [9]:
# Convertimos la respuesta en formato JSON a una estructura de Python
datos = response.json()

In [10]:
# Cargamos los datos en un DataFrame de pandas
df = pd.DataFrame(datos)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

In [12]:
# "aplanando" la estructura del JSON, transformándola en un DataFrame tabular (df_n) donde cada campo del JSON
 # (incluso los que están dentro de diccionarios o listas) se convierte en una columna.
df_aplanado = pd.json_normalize(datos)
df_aplanado.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [13]:
# Verificar la forma
df_aplanado.shape

(7267, 21)

In [14]:
# Verificar columnas del DataFrame
df_aplanado.columns

Index(['customerID', 'Churn', 'customer.gender', 'customer.SeniorCitizen',
       'customer.Partner', 'customer.Dependents', 'customer.tenure',
       'phone.PhoneService', 'phone.MultipleLines', 'internet.InternetService',
       'internet.OnlineSecurity', 'internet.OnlineBackup',
       'internet.DeviceProtection', 'internet.TechSupport',
       'internet.StreamingTV', 'internet.StreamingMovies', 'account.Contract',
       'account.PaperlessBilling', 'account.PaymentMethod',
       'account.Charges.Monthly', 'account.Charges.Total'],
      dtype='object')

In [15]:
#Explorar los tipos de datos y valores faltantes:
df_aplanado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

## Análisis de valores únicos por columna

Para entender mejor la diversidad y posibles categorías en cada variable, realizamos un conteo de valores únicos por columna. Esto ayuda a:

- Identificar variables categóricas con pocas categorías (por ejemplo, género, tipo de contrato).
- Detectar posibles columnas con muchos valores distintos que pueden requerir tratamiento especial (como IDs o valores numéricos continuos).
- Facilitar la inspección de las categorías presentes en variables relevantes, cuando su cantidad es manejable (menos de 50 valores únicos).

El siguiente código imprime la cantidad de valores únicos por columna, y muestra los valores concretos para aquellas con menos de 50 categorías.


In [16]:
# Mostrar la cantidad de valores únicos por columna
# e imprimir los valores únicos solo si son menos de 50 para mejor legibilidad
for col in df_aplanado.columns:
    num_unicos = df_aplanado[col].nunique()
    print(f"Columna '{col}' tiene {num_unicos} valores únicos.")

    if num_unicos < 50:
        print(f"Valores únicos en '{col}':")
        print(df_aplanado[col].unique())

    print('-' * 50)  # Línea divisoria para claridad visual


Columna 'customerID' tiene 7267 valores únicos.
--------------------------------------------------
Columna 'Churn' tiene 3 valores únicos.
Valores únicos en 'Churn':
['No' 'Yes' '']
--------------------------------------------------
Columna 'customer.gender' tiene 2 valores únicos.
Valores únicos en 'customer.gender':
['Female' 'Male']
--------------------------------------------------
Columna 'customer.SeniorCitizen' tiene 2 valores únicos.
Valores únicos en 'customer.SeniorCitizen':
[0 1]
--------------------------------------------------
Columna 'customer.Partner' tiene 2 valores únicos.
Valores únicos en 'customer.Partner':
['Yes' 'No']
--------------------------------------------------
Columna 'customer.Dependents' tiene 2 valores únicos.
Valores únicos en 'customer.Dependents':
['Yes' 'No']
--------------------------------------------------
Columna 'customer.tenure' tiene 73 valores únicos.
--------------------------------------------------
Columna 'phone.PhoneService' tiene 2 va

In [17]:
# Verificar si hay filas duplicadas
num_duplicados = df_aplanado.duplicated().sum()
print(f"Número de filas duplicadas: {num_duplicados}")

Número de filas duplicadas: 0


In [21]:
# Mostrar cantidad de valores nulos en todas las columnas
print("Cantidad de valores nulos por columna:")
print(df_aplanado.isnull().sum())

Cantidad de valores nulos por columna:
customerID                   0
Churn                        0
customer.gender              0
customer.SeniorCitizen       0
customer.Partner             0
customer.Dependents          0
customer.tenure              0
phone.PhoneService           0
phone.MultipleLines          0
internet.InternetService     0
internet.OnlineSecurity      0
internet.OnlineBackup        0
internet.DeviceProtection    0
internet.TechSupport         0
internet.StreamingTV         0
internet.StreamingMovies     0
account.Contract             0
account.PaperlessBilling     0
account.PaymentMethod        0
account.Charges.Monthly      0
account.Charges.Total        0
dtype: int64


In [22]:
# Contar celdas vacías o con solo espacios en blanco en cada columna
blancos = df_aplanado.apply(lambda x: x.astype(str).str.strip() == '').sum()

print("Cantidad de valores vacíos o en blanco por columna:")
print(blancos)


Cantidad de valores vacíos o en blanco por columna:
customerID                     0
Churn                        224
customer.gender                0
customer.SeniorCitizen         0
customer.Partner               0
customer.Dependents            0
customer.tenure                0
phone.PhoneService             0
phone.MultipleLines            0
internet.InternetService       0
internet.OnlineSecurity        0
internet.OnlineBackup          0
internet.DeviceProtection      0
internet.TechSupport           0
internet.StreamingTV           0
internet.StreamingMovies       0
account.Contract               0
account.PaperlessBilling       0
account.PaymentMethod          0
account.Charges.Monthly        0
account.Charges.Total         11
dtype: int64


In [23]:
# Convertir la columna 'account.Charges.Total' a tipo numérico (float)
# Los valores que no se puedan convertir quedarán como NaN (por ejemplo, valores vacíos o erróneos)
df_aplanado['account.Charges.Total'] = pd.to_numeric(df_aplanado['account.Charges.Total'], errors='coerce')

# Confirmamos el nuevo tipo de dato
print("Tipo de dato de 'account.Charges.Total':", df_aplanado['account.Charges.Total'].dtype)


Tipo de dato de 'account.Charges.Total': float64


In [24]:
# Eliminar filas donde la columna 'Churn' está vacía o solo con espacios
df_aplanado = df_aplanado[df_aplanado['Churn'].str.strip() != '']

print("Número de filas después de eliminar valores vacíos en 'Churn':", len(df_aplanado))


Número de filas después de eliminar valores vacíos en 'Churn': 7043


In [25]:
# Contar valores vacíos o en blanco (strings vacíos o solo espacios)
espacios_vacios = df_aplanado.apply(lambda x: x.astype(str).str.strip() == '').sum()

print("Cantidad de valores vacíos o en blanco por columna:")
print(espacios_vacios)


Cantidad de valores vacíos o en blanco por columna:
customerID                   0
Churn                        0
customer.gender              0
customer.SeniorCitizen       0
customer.Partner             0
customer.Dependents          0
customer.tenure              0
phone.PhoneService           0
phone.MultipleLines          0
internet.InternetService     0
internet.OnlineSecurity      0
internet.OnlineBackup        0
internet.DeviceProtection    0
internet.TechSupport         0
internet.StreamingTV         0
internet.StreamingMovies     0
account.Contract             0
account.PaperlessBilling     0
account.PaymentMethod        0
account.Charges.Monthly      0
account.Charges.Total        0
dtype: int64


In [26]:
df_aplanado.describe()

,customer.SeniorCitizen,customer.tenure,account.Charges.Monthly,account.Charges.Total
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [27]:
# Crear una nueva columna llamada 'Cuentas_Diarias'
# calculando el valor diario a partir del cargo mensual ('account.Charges.Monthly')
df_aplanado['Cuentas_Diarias'] = df_aplanado['account.Charges.Monthly'] / 30

# Mostrar las primeras filas de las columnas relevantes para verificar que el cálculo esté correcto
print(df_aplanado[['account.Charges.Monthly', 'Cuentas_Diarias']].head())


   account.Charges.Monthly  Cuentas_Diarias
0                     65.6         2.186667
1                     59.9         1.996667
2                     73.9         2.463333
3                     98.0         3.266667
4                     83.9         2.796667


#📊 Carga y análisis

#📄Informe final